# Setup, Imports & Data Load

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sqlite3
import math
from sklearn.preprocessing import LabelEncoder, MinMaxScaler, OrdinalEncoder, OneHotEncoder
from google.colab import drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Base de Datos de Clientes

In [ ]:
# df_cust_clean
df_cust_clean = pd.read_csv("/content/drive/MyDrive/TFM_EasyMoney/datasets_diccionario_TFM/df_cust_clean.csv")
df_cust_clean.head()

,pk_cid,active_customer,age,salary,short_term_deposit,loans,mortgage,funds,securities,long_term_deposit,...,entry_channel_KHK,entry_channel_KHM,entry_channel_KHN,entry_channel_KHQ,entry_channel_OTHERS,entry_channel_RED,grupo_edad_25-35,grupo_edad_35-45,grupo_edad_45-55,grupo_edad_55+
0,657826,1.0,44,54493.38,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
1,657817,0.0,32,103655.58,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2,657986,1.0,39,100993.17,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
3,657905,1.0,85,154059.09,0,0,0,0,0,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,657336,1.0,38,105118.08,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


## Data de Propensión de Compra

In [ ]:
# 1. Predicciones em_acount
df_prediccion_em_account = pd.read_csv("/content/drive/MyDrive/TFM_EasyMoney/datasets_diccionario_TFM/df_prediccion_em_account.csv")
df_prediccion_em_account.head()

,pk_id,predict,predict_proba
0,657817,0,0.000076
1,657986,0,0.013540
2,657147,0,0.063475
3,658184,0,0.064458
4,658576,1,0.681599


In [ ]:
# 2. Predicciones emc_acount
df_prediccion_emc_account = pd.read_csv("/content/drive/MyDrive/TFM_EasyMoney/datasets_diccionario_TFM/df_prediccion_emc_account.csv")
df_prediccion_emc_account.head()

,pk_id,predict,predict_proba
0,657826,1,0.951596
1,657817,0,0.000011
2,657905,1,0.804648
3,657336,0,0.013125
4,657147,1,0.712618


In [ ]:
# 3. Predicciones pension_plan
df_prediccion_pension_plan = pd.read_csv("/content/drive/MyDrive/TFM_EasyMoney/datasets_diccionario_TFM/df_prediccion_pension_plan.csv")
df_prediccion_pension_plan.head()

,pk_id,predict,predict_proba
0,657826,0.0,0.001726
1,657817,0.0,0.000114
2,657905,0.0,0.000363
3,657336,0.0,0.000326
4,657147,0.0,0.000440


In [ ]:
# 4. Predicciones long_term_deposit
df_prediccion_long_term_deposit = pd.read_csv("/content/drive/MyDrive/TFM_EasyMoney/datasets_diccionario_TFM/df_prediccion_long_term_deposit.csv")
df_prediccion_long_term_deposit.head()

,pk_id,predict,predict_proba
0,657826,1,0.607518
1,657817,0,0.000134
2,657986,1,0.676270
3,657336,0,0.010464
4,657147,0,0.002839


In [ ]:
# 5. Predicciones debit_card
df_prediccion_debit_card = pd.read_csv("/content/drive/MyDrive/TFM_EasyMoney/datasets_diccionario_TFM/df_prediccion_debit_card.csv")
df_prediccion_debit_card.head()

,pk_id,predict,predict_proba
0,657826,1,0.657267
1,657817,0,0.000061
2,657905,0,0.139143
3,657336,0,0.022380
4,657147,0,0.000618


## Data de Clusters

In [ ]:
# df_cluster
df_cluster = pd.read_csv("/content/drive/MyDrive/TFM_EasyMoney/datasets_diccionario_TFM/df_segmentos.csv")
df_cluster.head()

,pk_cid,cluster
0,657826,Pasivos Masivos
1,657817,Pasivos Masivos
2,657986,Pasivos Masivos
3,657905,Pasivos Masivos
4,657336,Pasivos Masivos


## Unificación Final de Data

In [ ]:
# --- A. Base oficial ---
curated_database = df_cust_clean.copy()

# Función auxiliar para formatear proba como porcentaje con 2 decimales
def format_percent(x):
    return (x * 100).round(2)

# --- B. Merge prediccion_em_account (M1) ---
curated_database = curated_database.merge(
    df_prediccion_em_account[['pk_id', 'predict_proba']].rename(columns={'predict_proba': 'M1'}),
    how='left',
    left_on='pk_cid',
    right_on='pk_id'
).drop(columns='pk_id')

# reemplazar nulos por 0 y formatear a porcentaje
curated_database['M1'] = format_percent(curated_database['M1'].fillna(0))

# --- C. Merge prediccion_emc_account (M2) ---
curated_database = curated_database.merge(
    df_prediccion_emc_account[['pk_id', 'predict_proba']].rename(columns={'predict_proba': 'M2'}),
    how='left',
    left_on='pk_cid',
    right_on='pk_id'
).drop(columns='pk_id')

curated_database['M2'] = format_percent(curated_database['M2'].fillna(0))

# --- D. Merge prediccion_pension_plan (M3) ---
curated_database = curated_database.merge(
    df_prediccion_pension_plan[['pk_id', 'predict_proba']].rename(columns={'predict_proba': 'M3'}),
    how='left',
    left_on='pk_cid',
    right_on='pk_id'
).drop(columns='pk_id')

curated_database['M3'] = format_percent(curated_database['M3'].fillna(0))

# --- E. Merge prediccion_long_term_deposit (M4) ---
curated_database = curated_database.merge(
    df_prediccion_long_term_deposit[['pk_id', 'predict_proba']].rename(columns={'predict_proba': 'M4'}),
    how='left',
    left_on='pk_cid',
    right_on='pk_id'
).drop(columns='pk_id')

curated_database['M4'] = format_percent(curated_database['M4'].fillna(0))

# --- F. Merge prediccion_debit_card (M5) ---
curated_database = curated_database.merge(
    df_prediccion_debit_card[['pk_id', 'predict_proba']].rename(columns={'predict_proba': 'M5'}),
    how='left',
    left_on='pk_cid',
    right_on='pk_id'
).drop(columns='pk_id')

curated_database['M5'] = format_percent(curated_database['M5'].fillna(0))

# --- G. Merge cluster (no debería haber incompletos) ---
curated_database = curated_database.merge(
    df_cluster[['pk_cid', 'cluster']],
    how='left',
    on='pk_cid'
)


In [ ]:
curated_database

,pk_cid,active_customer,age,salary,short_term_deposit,loans,mortgage,funds,securities,long_term_deposit,...,grupo_edad_25-35,grupo_edad_35-45,grupo_edad_45-55,grupo_edad_55+,M1,M2,M3,M4,M5,cluster
0,657826,1.0,44,54493.380,0,0,0,0,0,0,...,0.0,1.0,0.0,0.0,0.00,95.16,0.17,60.75,65.73,Pasivos Masivos
1,657817,0.0,32,103655.580,0,0,0,0,0,0,...,1.0,0.0,0.0,0.0,0.01,0.00,0.01,0.01,0.01,Pasivos Masivos
2,657986,1.0,39,100993.170,0,0,0,0,0,0,...,0.0,1.0,0.0,0.0,1.35,0.00,0.00,67.63,0.00,Pasivos Masivos
3,657905,1.0,85,154059.090,0,0,0,0,0,1,...,0.0,0.0,0.0,1.0,0.00,80.46,0.04,0.00,13.91,Pasivos Masivos
4,657336,1.0,38,105118.080,0,0,0,0,0,0,...,0.0,1.0,0.0,0.0,0.00,1.31,0.03,1.05,2.24,Pasivos Masivos
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
440239,1166765,0.0,22,43912.170,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.00,0.02,0.00,0.00,0.05,Pasivos Masivos
440240,1166764,0.0,23,23334.990,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.00,0.20,0.00,0.01,32.14,Pasivos Masivos
440241,1166763,1.0,47,77366.970,0,0,0,0,0,0,...,0.0,0.0,1.0,0.0,0.00,7.30,0.16,8.19,25.15,Pasivos Masivos
440242,1166789,0.0,22,199592.820,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.00,0.04,0.00,0.01,0.02,Pasivos Masivos


In [ ]:
curated_database.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 440244 entries, 0 to 440243
Data columns (total 50 columns):
 #   Column                      Non-Null Count   Dtype  
---  ------                      --------------   -----  
 0   pk_cid                      440244 non-null  int64  
 1   active_customer             440244 non-null  float64
 2   age                         440244 non-null  int64  
 3   salary                      440244 non-null  float64
 4   short_term_deposit          440244 non-null  int64  
 5   loans                       440244 non-null  int64  
 6   mortgage                    440244 non-null  int64  
 7   funds                       440244 non-null  int64  
 8   securities                  440244 non-null  int64  
 9   long_term_deposit           440244 non-null  int64  
 10  em_account_pp               440244 non-null  int64  
 11  credit_card                 440244 non-null  int64  
 12  payroll                     440244 non-null  float64
 13  pension_plan  

In [ ]:
curated_database.to_csv('curated_database.csv', index=False)

# Propuesta Comercial

## Supuestos y Cálculo de Costes

En los close rates estimados se calcula un porcentaje mucho menor de conversión para campañas masivas de email marketing debido a que son acciones masivas menos personalizadas y dedicadas al cliente a diferencia de una llamada telefónica.

In [ ]:
# Hipótesis de estimaciones del Close Rate

#Campañas de email marketing o push

cr_p_email = 0.003 # Escenario pesimista 0.3%
cr_r_email = 0.008 # Escenario real probable 0.8%
cr_o_email = 0.015 # Escenario optimista 1.5%

#Campañas de televenta personalizadas

cr_p_telf = 0.03 # Escenario pesimista 3%
cr_r_telf = 0.08 # Escenario real probable 8%
cr_o_telf = 0.15 # Escenario optimista 15%

In [ ]:
# Net margin promedio por producto
net_margin = {
    "em_acount": 70,
    "emc_account": 70,
    "pension_plan": 5976,
    "long_term_deposit": 1497,
    "debit_card": 60
}

In [ ]:
# COSTOS CAMPAÑAS DE EMAIL MASIVOS

def email_mkt_cost(clientes_contactables):

    costo_por_envio = 0.004  # € por email enviado y gestión
    costo_crm = 300         # € herramienta (Mailchimp, CRM)
    costo_diseño = 300      # € diseño, contenido, copy
    costo_gestion = 1000    # € prorrateo de tiempo dedicado Analista de CRM
    costo_fijo = costo_crm + costo_diseño + costo_gestion

    costo_variable = clientes_contactables * costo_por_envio
    costo_total = costo_fijo + costo_variable

    return round(costo_total,2)

In [ ]:
# COSTOS CAMPAÑAS DE TELEVENTA

def cold_call_cost(clientes_contactables):

    llamadas_efectivas = clientes_contactables * 0.45 # se espera un % de efectividad de 45%
    nr_asesores = math.ceil(llamadas_efectivas / 1550)           # se espera un asesor tiene capacidad de realizar 1550 llamadas efectivas mensuales

    salario_asesor = 1400 * nr_asesores                     # € salario mensual asesor
    seguridad_social = salario_asesor * 0.32  # € impuestos a la seguridad social del empleado
    capacitacion_comercial = 1000             # € costo unico
    gestion = 300                           # € costes de softwares de gestion y administrativos

    costo_fijo = (salario_asesor + seguridad_social + capacitacion_comercial + gestion)

    costo_por_llamada = 0.5                 # € por llamada efectiva (promedio de 1 minuto)
    llamadas_efectivas = clientes_contactables * 0.45 # se espera un % de efectividad de 45%
    costo_variable = costo_por_llamada * llamadas_efectivas

    costo_total = costo_fijo + costo_variable

    return round(costo_total,2)

In [ ]:
curated_database['cluster'].value_counts()

,count
cluster,
Pasivos Masivos,413087
Clientes Vinculados Premium,8982
Seniors Conservadores,8469
Multivinculados Endeudados,7254
Deudores Rentables,2452


## Campaña 1 - Pasivos Masivos: Reactivación de Clientes

A continuacióon se estima la cantidad de clientes impactados a partir de ofrecer cualquiera de los productos de baja rentabilidad a partir de una campaña de reactivación por email con la promoción de reactivar el uso de la cuenta mediante un incentivo de cashback de 10 euros (posteriormente incluidos en los costos de la adquisición).

Se dividen en dos subsegmentos de clientes puesto que el net_margin promedio generado por cada producto es distinto y se debe dividir para una estimación de beneficios totales más precisa.

**Producto/s ofrecido/s: em_acount, emc_account y/o debit_card**

**Foco principal de la campaña:** Reincorporar clientes inactivos mediante la reactivación de productos base, incrementando la tasa de vinculación y flujo de efectivo.

**Nombre de la campaña:** Reactiva tu cuenta.

**Comunicación al cliente final:** Vuelve a disfrutar de los beneficios de tu cuenta activa: transferencias, pagos y gestión 100% online sin comisiones ocultas con una bonificación de 10 euros.

In [ ]:
# Base de clientes que comprarían em_acount o debit_card ---> (Cuenta corriente o crypto)

campaign_base_1a = curated_database[
    (curated_database['cluster'] == 'Pasivos Masivos') &
    (
        ((curated_database['em_acount'] == 0) & (curated_database['M1'] >= 0.85)) |
        ((curated_database['emc_account'] == 0) & (curated_database['M2'] >= 0.85))
    )
].shape[0]
campaign_base_1a

192147

In [ ]:
# Base de clientes que comprarían debit card ---> (Tarjeta de débito)

campaign_base_1b = (
    curated_database[
    (curated_database['cluster'] == 'Pasivos Masivos') &
    (
        ((curated_database['em_acount'] == 0) & (curated_database['M1'] >= 0.85)) |
        ((curated_database['debit_card'] == 0) & (curated_database['M5'] >= 0.85)) |
        ((curated_database['emc_account'] == 0) & (curated_database['M2'] >= 0.85))
    )
].shape[0]
                    ) - campaign_base_1a
campaign_base_1b

21056

In [ ]:
campaign_base_1 = campaign_base_1a + campaign_base_1b
campaign_base_1

213203

Una vez identificada la cantidad total de clientes a contactar para esta campaña (210.432 clientes), se procede a calcular el coste de la campaña total a partir de los supuestos definidos en la función anterior a partir de una acción de email MKT.

In [ ]:
campaign_cost_1 = email_mkt_cost(campaign_base_1)
campaign_cost_1

2452.81

A continuación se calcula beneficio esperado por la campaña bajo los supuestos de close rate y los 3 potenciales escenarios, para calcular la cantidad de clientes que pueden convertir y el beneficio final de la campaña después de deducir los costes incurridos.

### Conversión de Clientes

In [ ]:
# Escenario pesimista de clientes convertidos

acquired_customer_p_1 = round(campaign_base_1 * cr_p_email,0)
acquired_customer_p_1

640.0

In [ ]:
# Escenario real de clientes convertidos

acquired_customer_r_1 = round(campaign_base_1 * cr_r_email,0)
acquired_customer_r_1

1706.0

In [ ]:
# Escenario optimista de clientes convertidos

acquired_customer_o_1 = round(campaign_base_1 * cr_o_email,0)
acquired_customer_o_1

3198.0

### Beneficios de la Campaña

Al ser esta una campaña masiva donde el Call To Action (CTA) para que los clientes se reactiven es una promoción de cashback "Reactiva tu uso en la cuenta..." o "Pide ya tu tarjeta de débito..." el gancho de la campaña es regalar 10€ por la acción para captar al cliente. Este "costo" es asumido directamente restando el net_margin por producto puesto que únicamente se incurre al momento de la conversión.

In [ ]:
# Escenario pesimista de beneficios de la campaña

net_margin_p_1 = (
    (round(campaign_base_1a * cr_p_email,0) * (net_margin["em_acount"] - 10)) # Se restan los 10€ de incentivo al net_margin del prod
    + (round(campaign_base_1b * cr_p_email,0) * (net_margin["debit_card"] - 10))
)


campaign_benefit_p_1 = net_margin_p_1 - campaign_cost_1 - (10 * acquired_customer_p_1)


campaign_roi_p_1 = campaign_benefit_p_1 / (campaign_cost_1 + (10 * acquired_customer_p_1))

campaign_cost_p_1 = net_margin_p_1 - campaign_benefit_p_1

print(f"Resumen Pesimista de la Campaña 1:\n")
print(f"Clientes Convertidos: {int(acquired_customer_p_1)}")
print(f"Valor generado: {net_margin_p_1}")
print(f"Costos Incurridos: {campaign_cost_p_1}")
print(f"Beneficios netos generados: {campaign_benefit_p_1}")
print(f"ROI: {campaign_roi_p_1 * 100:.2f}%")

Resumen Pesimista de la Campaña 1:

Clientes Convertidos: 640
Valor generado: 37710.0
Costos Incurridos: 8852.809999999998
Beneficios netos generados: 28857.190000000002
ROI: 325.97%


In [ ]:
# Escenario real de beneficios de la campaña

net_margin_r_1 = (
    (round(campaign_base_1a * cr_r_email,0) * (net_margin["em_acount"] - 10)) # Se restan los 10€ de incentivo al net_margin del prod
    + (round(campaign_base_1b * cr_r_email,0) * (net_margin["debit_card"] - 10))
)

campaign_benefit_r_1 = net_margin_r_1 - campaign_cost_1 - (10 * acquired_customer_r_1)

campaign_roi_r_1 = campaign_benefit_r_1 / (campaign_cost_1 + (10 * acquired_customer_r_1))

campaign_cost_r_1 = net_margin_r_1 - campaign_benefit_r_1

print(f"Resumen Real de la Campaña 1:\n")
print(f"Clientes Convertidos: {int(acquired_customer_r_1)}")
print(f"Costos Incurridos: {campaign_cost_r_1}")
print(f"Valor generado: {net_margin_r_1}")
print(f"Beneficios netos generados: {campaign_benefit_r_1}")
print(f"ROI: {campaign_roi_r_1 * 100:.2f}%")

Resumen Real de la Campaña 1:

Clientes Convertidos: 1706
Costos Incurridos: 19512.809999999998
Valor generado: 100620.0
Beneficios netos generados: 81107.19
ROI: 415.66%


In [ ]:
# Escenario optimista de beneficios de la campaña

net_margin_o_1 = (
    (round(campaign_base_1a * cr_o_email,0) * (net_margin["em_acount"] - 10)) # Se restan los 10€ de incentivo al net_margin del prod
    + (round(campaign_base_1b * cr_o_email,0) * (net_margin["debit_card"] - 10))
)

campaign_benefit_o_1 = net_margin_o_1 - campaign_cost_1 - (10 * acquired_customer_o_1)

campaign_roi_o_1 = campaign_benefit_o_1 / (campaign_cost_1 + (10 * acquired_customer_o_1))

campaign_cost_o_1 = net_margin_o_1 - campaign_benefit_o_1

print(f"Resumen Optimista de la Campaña 1:\n")
print(f"Clientes Convertidos: {int(acquired_customer_o_1)}")
print(f"Costos Incurridos: {campaign_cost_o_1}")
print(f"Valor generado: {net_margin_o_1}")
print(f"Beneficios netos generados: {campaign_benefit_o_1}")
print(f"ROI: {campaign_roi_o_1 * 100:.2f}%")

Resumen Optimista de la Campaña 1:

Clientes Convertidos: 3198
Costos Incurridos: 34432.81
Valor generado: 188720.0
Beneficios netos generados: 154287.19
ROI: 448.08%


## Campaña 2 - Clientes Vinculados Premium: Rentabilizar Cartera

Para la 2da campaña enfocada a los Clientes Vinculados Premium, se busca realizar una campaña de telemarketing dedicada con el objetivo de rentabilizar el segmento a partir de ofrecer un producto de alta rentabilidad.

**Producto ofrecido:** pension_plan

**Foco principal de la campaña:** Profundizar la rentabilidad de clientes de alto valor con productos de largo plazo que aumenten el lifetime value y la fidelización.

**Nombre de la campaña:** Planifica tu futuro hoy.

**Comunicación al cliente final:** Haz crecer tu patrimonio con un plan de jubilación flexible y beneficios fiscales diseñados para tu perfil.

In [ ]:
# 2 Campaña Vinculados Premium
campaign_base_2 = curated_database[(curated_database['cluster'] == 'Clientes Vinculados Premium') & (curated_database['pension_plan'] == 0)& (curated_database['M3'] >= 0.85)].shape[0]
campaign_base_2

3884

Una vez identificada la cantidad total de clientes a contactar para esta campaña (3.880 clientes), se procede a calcular el coste de la campaña total a partir de los supuestos definidos en la función anterior a partir de una acción de telemarketing.

In [ ]:
campaign_cost_2 = cold_call_cost(campaign_base_2)
campaign_cost_2

5869.9

Considerando la demanda de **1 asesor** de televentas para este caso.

A continuación se calcula beneficio esperado por la campaña bajo los supuestos de close rate y los 3 potenciales escenarios, para calcular la cantidad de clientes que pueden convertir y el beneficio final de la campaña después de deducir los costes incurridos.

### Conversión de Clientes

In [ ]:
# Escenario pesimista de clientes convertidos

acquired_customer_p_2 = round(campaign_base_2 * cr_p_telf,0)
acquired_customer_p_2

117.0

In [ ]:
# Escenario real de clientes convertidos

acquired_customer_r_2 = round(campaign_base_2 * cr_r_telf,0)
acquired_customer_r_2

311.0

In [ ]:
# Escenario optimista de clientes convertidos

acquired_customer_o_2 = round(campaign_base_2 * cr_o_telf,0)
acquired_customer_o_2

583.0

### Beneficios de la Campaña

A continuación se procede a calcular el beneficio por la campaña para cada escenario, se considera un porcentaje de 15% sobre el net_margin promedio por conversión que sería atribuido como parte de comisiones de venta para el agente de televenta.

In [ ]:
# Escenario pesimista de beneficios de la campaña

net_margin_p_2 = round((campaign_base_2 * cr_p_telf) * (net_margin["pension_plan"] * 0.85),2) # Se deduce el 15% por concepto de comisión de vtas

campaign_benefit_p_2 = net_margin_p_2 - campaign_cost_2 - ((net_margin["pension_plan"] * 0.15) * acquired_customer_p_2)

campaign_roi_p_2 = campaign_benefit_p_2 / (campaign_cost_2 + ((net_margin["pension_plan"] * 0.15) * acquired_customer_p_2))

campaign_cost_p_2 = net_margin_p_2 - campaign_benefit_p_2

print(f"Resumen Pesimista de la Campaña 2:\n")
print(f"Clientes Convertidos: {int(acquired_customer_p_2)}")
print(f"Valor generado: {net_margin_p_2}")
print(f"Costos Incurridos: {campaign_cost_p_2}")
print(f"Beneficios netos generados: {campaign_benefit_p_2}")
print(f"ROI: {campaign_roi_p_2 * 100:.2f}%")

Resumen Pesimista de la Campaña 2:

Clientes Convertidos: 117
Valor generado: 591874.99
Costos Incurridos: 110748.70000000001
Beneficios netos generados: 481126.29
ROI: 434.43%


In [ ]:
# Escenario realista de beneficios de la campaña

net_margin_r_2 = round((campaign_base_2 * cr_r_telf) * (net_margin["pension_plan"] * 0.85),2) # Se deduce el 15% por concepto de comisión de vtas

campaign_benefit_r_2 = net_margin_r_2 - campaign_cost_2 - ((net_margin["pension_plan"] * 0.15) * acquired_customer_r_2)

campaign_roi_r_2 = campaign_benefit_r_2 / (campaign_cost_2 + ((net_margin["pension_plan"] * 0.15) * acquired_customer_r_2))

campaign_cost_r_2 = net_margin_r_2 - campaign_benefit_r_2

print(f"Resumen Real de la Campaña 2:\n")
print(f"Clientes Convertidos: {int(acquired_customer_r_2)}")
print(f"Valor generado: {net_margin_r_2}")
print(f"Costos Incurridos: {campaign_cost_r_2}")
print(f"Beneficios netos generados: {campaign_benefit_r_2}")
print(f"ROI: {campaign_roi_r_2 * 100:.2f}%")

Resumen Real de la Campaña 2:

Clientes Convertidos: 311
Costos Incurridos: 284650.2999999998
Valor generado: 1578333.31
Beneficios netos generados: 1293683.0100000002
ROI: 454.48%


In [ ]:
# Escenario optimista de beneficios de la campaña

net_margin_o_2 = round((campaign_base_2 * cr_o_telf) * (net_margin["pension_plan"] * 0.85),2) # Se deduce el 15% por concepto de comisión de vtas

campaign_benefit_o_2 = net_margin_o_2 - campaign_cost_2 - ((net_margin["pension_plan"] * 0.15) * acquired_customer_o_2)

campaign_roi_o_2 = campaign_benefit_o_2 / (campaign_cost_2 + ((net_margin["pension_plan"] * 0.15) * acquired_customer_o_2))

campaign_cost_o_2 = net_margin_o_2 - campaign_benefit_o_2

print(f"Resumen Optimista de la Campaña 2:\n")
print(f"Clientes Convertidos: {int(acquired_customer_o_2)}")
print(f"Valor generado: {net_margin_o_2}")
print(f"Costos Incurridos: {campaign_cost_o_2}")
print(f"Beneficios netos generados: {campaign_benefit_o_2}")
print(f"ROI: {campaign_roi_o_2 * 100:.2f}%")

Resumen Optimista de la Campaña 2:

Clientes Convertidos: 583
Valor generado: 2959374.96
Costos Incurridos: 528471.1000000001
Beneficios netos generados: 2430903.86
ROI: 459.99%


## Campaña 3 - Deudores Rentables: Rentabilizar Cartera

Para la 3ra campaña enfocada a los Deudores Rentables, se busca realizar una campaña de telemarketing dedicada con el objetivo de rentabilizar el segmento a partir de ofrecer un producto de alta rentabilidad.

**Producto ofrecido:** long_term_deposit

**Foco principal de la campaña:** Mantener el valor del segmento reduciendo el riesgo de impago, incentivando la migración hacia productos de ahorro seguro.

**Nombre de la campaña:** Rentabilidad con equilibrio.

**Comunicación al cliente final:** Convierte tus ingresos en estabilidad: un depósito que te asegura rendimientos fijos sin sobresaltos.

In [ ]:
# 3 Campaña Deudores Rentables
campaign_base_3 = curated_database[(curated_database['cluster'] == 'Deudores Rentables') & (curated_database['long_term_deposit'] == 0)& (curated_database['M4'] >= 0.85)].shape[0]
campaign_base_3

2284

Una vez identificada la cantidad total de clientes a contactar para esta campaña (2.276 clientes), se procede a calcular el coste de la campaña total a partir de los supuestos definidos en la función anterior a partir de una acción de telemarketing.

In [ ]:
campaign_cost_3 = cold_call_cost(campaign_base_3)
campaign_cost_3

3661.9

### Conversión de Clientes

In [ ]:
# Escenario pesimista de clientes convertidos

acquired_customer_p_3 = round(campaign_base_3 * cr_p_telf,0)
acquired_customer_p_3

69.0

In [ ]:
# Escenario real de clientes convertidos

acquired_customer_r_3 = round(campaign_base_3 * cr_r_telf,0)
acquired_customer_r_3

183.0

In [ ]:
# Escenario optimista de clientes convertidos

acquired_customer_o_3 = round(campaign_base_3 * cr_o_telf,0)
acquired_customer_o_3

343.0

### Beneficios de la Campaña

A continuación se procede a calcular el beneficio por la campaña para cada escenario, se considera un porcentaje de 15% sobre el net_margin promedio por conversión que sería atribuido como parte de comisiones de venta para el agente de televenta.

In [ ]:
# Escenario pesimista de beneficios de la campaña

net_margin_p_3 = round((campaign_base_3 * cr_p_telf) * (net_margin["long_term_deposit"] * 0.85),2) # Se deduce el 15% por concepto de comisión de vtas

campaign_benefit_p_3 = net_margin_p_3 - campaign_cost_3 - ((net_margin["long_term_deposit"] * 0.15) * acquired_customer_p_3)

campaign_roi_p_3 = campaign_benefit_p_3 / (campaign_cost_3 + ((net_margin["long_term_deposit"] * 0.15) * acquired_customer_p_3))

campaign_cost_p_3 = net_margin_p_3 - campaign_benefit_p_3

print(f"Resumen Pesimista de la Campaña 3:\n")
print(f"Clientes Convertidos: {int(acquired_customer_p_3)}")
print(f"Costos Incurridos: {campaign_cost_p_3}")
print(f"Valor generado: {net_margin_p_3}")
print(f"Beneficios netos generados: {campaign_benefit_p_3}")
print(f"ROI: {campaign_roi_p_3 * 100:.2f}%")

Resumen Pesimista de la Campaña 3:

Clientes Convertidos: 69
Costos Incurridos: 19155.84999999999
Valor generado: 87188.27
Beneficios netos generados: 68032.42000000001
ROI: 355.15%


In [ ]:
# Escenario realista de beneficios de la campaña

net_margin_r_3 = round((campaign_base_3 * cr_r_telf) * (net_margin["long_term_deposit"] * 0.85),2) # Se deduce el 15% por concepto de comisión de vtas

campaign_benefit_r_3 = net_margin_r_3 - campaign_cost_3 - ((net_margin["long_term_deposit"] * 0.15) * acquired_customer_r_3)

campaign_roi_r_3 = campaign_benefit_r_3 / (campaign_cost_3 + ((net_margin["long_term_deposit"] * 0.15) * acquired_customer_r_3))

campaign_cost_r_3 = net_margin_r_3 - campaign_benefit_r_3

print(f"Resumen Real de la Campaña 3:\n")
print(f"Clientes Convertidos: {int(acquired_customer_r_3)}")
print(f"Valor generado: {net_margin_r_3}")
print(f"Costos Incurridos: {campaign_cost_r_3}")
print(f"Beneficios netos generados: {campaign_benefit_r_3}")
print(f"ROI: {campaign_roi_r_3 * 100:.2f}%")

Resumen Real de la Campaña 3:

Clientes Convertidos: 183
Valor generado: 232502.06
Costos Incurridos: 44754.54999999999
Beneficios netos generados: 187747.51
ROI: 419.50%


In [ ]:
# Escenario optimista de beneficios de la campaña

net_margin_o_3 = round((campaign_base_3 * cr_o_telf) * (net_margin["long_term_deposit"] * 0.85),2) # Se deduce el 15% por concepto de comisión de vtas

campaign_benefit_o_3 = net_margin_o_3 - campaign_cost_3 - ((net_margin["long_term_deposit"] * 0.15) * acquired_customer_o_3)

campaign_roi_o_3 = campaign_benefit_o_3 / (campaign_cost_3 + ((net_margin["long_term_deposit"] * 0.15) * acquired_customer_o_3))

campaign_cost_o_3 = net_margin_o_3 - campaign_benefit_o_3

print(f"Resumen Optimista de la Campaña 3:\n")
print(f"Clientes Convertidos: {int(acquired_customer_o_3)}")
print(f"Valor generado: {net_margin_o_3}")
print(f"Costos Incurridos: {campaign_cost_o_3}")
print(f"Beneficios netos generados: {campaign_benefit_o_3}")
print(f"ROI: {campaign_roi_o_3 * 100:.2f}%")

Resumen Optimista de la Campaña 3:

Clientes Convertidos: 343
Valor generado: 435941.37
Costos Incurridos: 80682.55000000005
Beneficios netos generados: 355258.81999999995
ROI: 440.32%


## Campaña 4 - Seniors Conservadores: Rentabilizar Cartera

Para la 4ta campaña enfocada a los Seniors Conservadores, se busca realizar una campaña de telemarketing dedicada con el objetivo de rentabilizar el segmento a partir de ofrecer un producto de alta rentabilidad.

**Producto ofrecido:** pension_plan

**Foco principal de la campaña:** Consolidar relaciones de largo plazo mediante productos previsionales, reforzando la imagen de la entidad como aliada de la jubilación segura.

**Nombre de la campaña:** Tranquilidad Garantizada.

**Comunicación al cliente final:** Asegura tu futuro con un plan pensado para darte tranquilidad y rentabilidad estable.

In [ ]:
# 4 Campaña Seniors Conservadores
campaign_base_4 = curated_database[(curated_database['cluster'] == 'Seniors Conservadores') & (curated_database['pension_plan'] == 0)& (curated_database['M3'] >= 0.85)].shape[0]
campaign_base_4

3900

Una vez identificada la cantidad total de clientes a contactar para esta campaña (3.758 clientes), se procede a calcular el coste de la campaña total a partir de los supuestos definidos en la función anterior a partir de una acción de telemarketing.

In [ ]:
campaign_cost_4 = cold_call_cost(campaign_base_4)
campaign_cost_4

5873.5

### Conversión de Clientes

In [ ]:
# Escenario pesimista de clientes convertidos

acquired_customer_p_4 = round(campaign_base_4 * cr_p_telf,0)
acquired_customer_p_4

117.0

In [ ]:
# Escenario real de clientes convertidos

acquired_customer_r_4 = round(campaign_base_4 * cr_r_telf,0)
acquired_customer_r_4

312.0

In [ ]:
# Escenario optimista de clientes convertidos

acquired_customer_o_4 = round(campaign_base_4 * cr_o_telf,0)
acquired_customer_o_4

585.0

### Beneficios de la Campaña

A continuación se procede a calcular el beneficio por la campaña para cada escenario, se considera un porcentaje de 15% sobre el net_margin promedio por conversión que sería atribuido como parte de comisiones de venta para el agente de televenta.

In [ ]:
# Escenario pesimista de beneficios de la campaña

net_margin_p_4 = round((campaign_base_4 * cr_p_telf) * (net_margin["pension_plan"] * 0.85),2) # Se deduce el 15% por concepto de comisión de vtas

campaign_benefit_p_4 = net_margin_p_4 - campaign_cost_4 - ((net_margin["pension_plan"] * 0.15) * acquired_customer_p_4)

campaign_roi_p_4 = campaign_benefit_p_4 / (campaign_cost_4 + ((net_margin["pension_plan"] * 0.15) * acquired_customer_p_4))

campaign_cost_p_4 = net_margin_p_4 - campaign_benefit_p_4

print(f"Resumen Pesimista de la Campaña 4:\n")
print(f"Clientes Convertidos: {int(acquired_customer_p_4)}")
print(f"Valor generado: {net_margin_p_4}")
print(f"Costos Incurridos: {campaign_cost_p_4}")
print(f"Beneficios netos generados: {campaign_benefit_p_4}")
print(f"ROI: {campaign_roi_p_4 * 100:.2f}%")

Resumen Pesimista de la Campaña 4:

Clientes Convertidos: 117
Valor generado: 594313.2
Costos Incurridos: 110752.29999999999
Beneficios netos generados: 483560.89999999997
ROI: 436.61%


In [ ]:
# Escenario realista de beneficios de la campaña

net_margin_r_4 = round((campaign_base_4 * cr_r_telf) * (net_margin["pension_plan"] * 0.85),2) # Se deduce el 15% por concepto de comisión de vtas

campaign_benefit_r_4 = net_margin_r_4 - campaign_cost_4 - ((net_margin["pension_plan"] * 0.15) * acquired_customer_r_4)

campaign_roi_r_4 = campaign_benefit_r_4 / (campaign_cost_4 + ((net_margin["pension_plan"] * 0.15) * acquired_customer_r_4))

campaign_cost_r_4 = net_margin_r_4 - campaign_benefit_r_4

print(f"Resumen Real de la Campaña 4:\n")
print(f"Clientes Convertidos: {int(acquired_customer_r_4)}")
print(f"Valor generado: {net_margin_r_4}")
print(f"Costos Incurridos: {campaign_cost_r_4}")
print(f"Beneficios netos generados: {campaign_benefit_r_4}")
print(f"ROI: {campaign_roi_r_4 * 100:.2f}%")

Resumen Real de la Campaña 4:

Clientes Convertidos: 312
Valor generado: 1584835.2
Costos Incurridos: 285550.30000000005
Beneficios netos generados: 1299284.9
ROI: 455.01%


In [ ]:
# Escenario optimista de beneficios de la campaña

net_margin_o_4 = round((campaign_base_4 * cr_o_telf) * (net_margin["pension_plan"] * 0.85),2) # Se deduce el 15% por concepto de comisión de vtas

campaign_benefit_o_4 = net_margin_o_4 - campaign_cost_4 - ((net_margin["pension_plan"] * 0.15) * acquired_customer_o_4)

campaign_roi_o_4 = campaign_benefit_o_4 / (campaign_cost_4 + ((net_margin["pension_plan"] * 0.15) * acquired_customer_o_4))

campaign_cost_o_4 = net_margin_o_4 - campaign_benefit_o_4

print(f"Resumen Optimista de la Campaña 4:\n")
print(f"Clientes Convertidos: {int(acquired_customer_o_4)}")
print(f"Valor generado: {net_margin_o_4}")
print(f"Costos Incurridos: {campaign_cost_o_4}")
print(f"Beneficios netos generados: {campaign_benefit_o_4}")
print(f"ROI: {campaign_roi_o_4 * 100:.2f}%")

Resumen Optimista de la Campaña 4:

Clientes Convertidos: 585
Valor generado: 2971566.0
Costos Incurridos: 530267.5
Beneficios netos generados: 2441298.5
ROI: 460.39%


## Campaña 5 - Multivinculados Endeudados: Rentabilizar Cartera

Para la última campaña enfocada a los Multivinculados eNDEUDADOS, se busca realizar una campaña de telemarketing dedicada con el objetivo de rentabilizar el segmento a partir de ofrecer un producto de alta rentabilidad.

**Producto ofrecido:** long_term_deposit

**Foco principal de la campaña:** Mejorar la gestión de liquidez de clientes endeudados, ayudándolos a equilibrar su cartera sin perder rentabilidad.

**Nombre de la campaña:** Tu Ahorro Inteligente.

**Comunicación al cliente final:** Optimiza tus finanzas con un depósito que te permite ganar más mientras reduces tu carga financiera.

In [ ]:
# 5 Campaña Multivinculados Endeudados
campaign_base_5 = curated_database[(curated_database['cluster'] == 'Multivinculados Endeudados') & (curated_database['long_term_deposit'] == 0)& (curated_database['M4'] >= 0.85)].shape[0]
campaign_base_5

6897

Una vez identificada la cantidad total de clientes a contactar para esta campaña (6.862 clientes), se procede a calcular el coste de la campaña total a partir de los supuestos definidos en la función anterior a partir de una acción de telemarketing.

In [ ]:
campaign_cost_5 = cold_call_cost(campaign_base_5)
campaign_cost_5

8395.83

### Conversión de Clientes

In [ ]:
# Escenario pesimista de clientes convertidos

acquired_customer_p_5 = round(campaign_base_5 * cr_p_telf,0)
acquired_customer_p_5

207.0

In [ ]:
# Escenario real de clientes convertidos

acquired_customer_r_5 = round(campaign_base_5 * cr_r_telf,0)
acquired_customer_r_5

552.0

In [ ]:
# Escenario optimista de clientes convertidos

acquired_customer_o_5 = round(campaign_base_5 * cr_o_telf,0)
acquired_customer_o_5

1035.0

### Beneficios de la Campaña

A continuación se procede a calcular el beneficio por la campaña para cada escenario, se considera un porcentaje de 15% sobre el net_margin promedio por conversión que sería atribuido como parte de comisiones de venta para el agente de televenta.

In [ ]:
# Escenario pesimista de beneficios de la campaña

net_margin_p_5 = round((campaign_base_5 * cr_p_telf) * (net_margin["long_term_deposit"] * 0.85),2) # Se deduce el 15% por concepto de comisión de vtas

campaign_benefit_p_5 = net_margin_p_5 - campaign_cost_5 - ((net_margin["long_term_deposit"] * 0.15) * acquired_customer_p_5)

campaign_roi_p_5 = campaign_benefit_p_5 / (campaign_cost_5 + ((net_margin["long_term_deposit"] * 0.15) * acquired_customer_p_5))

campaign_cost_p_5 = net_margin_p_5 - campaign_benefit_p_5

print(f"Resumen Pesimista de la Campaña 5:\n")
print(f"Clientes Convertidos: {int(acquired_customer_p_5)}")
print(f"Costos Incurridos: {campaign_cost_p_5}")
print(f"Valor generado: {net_margin_p_5}")
print(f"Beneficios netos generados: {campaign_benefit_p_5}")
print(f"ROI: {campaign_roi_p_5 * 100:.2f}%")

Resumen Pesimista de la Campaña 5:

Clientes Convertidos: 207
Costos Incurridos: 54877.67999999999
Valor generado: 263282.63
Beneficios netos generados: 208404.95
ROI: 379.76%


In [ ]:
# Escenario realista de beneficios de la campaña

net_margin_r_5 = round((campaign_base_5 * cr_r_telf) * (net_margin["long_term_deposit"] * 0.85),2) # Se deduce el 15% por concepto de comisión de vtas

campaign_benefit_r_5 = net_margin_r_5 - campaign_cost_5 - ((net_margin["long_term_deposit"] * 0.15) * acquired_customer_r_5)

campaign_roi_r_5 = campaign_benefit_r_5 / (campaign_cost_5 + ((net_margin["long_term_deposit"] * 0.15) * acquired_customer_r_4))

campaign_cost_r_5 = net_margin_r_5 - campaign_benefit_r_5

print(f"Resumen Real de la Campaña 5:\n")
print(f"Clientes Convertidos: {int(acquired_customer_r_5)}")
print(f"Valor generado: {net_margin_r_5}")
print(f"Costos Incurridos: {campaign_cost_r_5}")
print(f"Beneficios netos generados: {campaign_benefit_r_5}")
print(f"ROI: {campaign_roi_r_5 * 100:.2f}%")

Resumen Real de la Campaña 5:

Clientes Convertidos: 552
Valor generado: 702087.01
Costos Incurridos: 132347.42999999993
Beneficios netos generados: 569739.5800000001
ROI: 726.20%


In [ ]:
# Escenario optimista de beneficios de la campaña

net_margin_o_5 = round((campaign_base_5 * cr_o_telf) * (net_margin["long_term_deposit"] * 0.85),2) # Se deduce el 15% por concepto de comisión de vtas

campaign_benefit_o_5 = net_margin_o_5 - campaign_cost_5 - ((net_margin["long_term_deposit"] * 0.15) * acquired_customer_o_5)

campaign_roi_o_5 = campaign_benefit_o_5 / (campaign_cost_5 + ((net_margin["long_term_deposit"] * 0.15) * acquired_customer_o_5))

campaign_cost_o_5 = net_margin_o_5 - campaign_benefit_o_5

print(f"Resumen Optimista de la Campaña 5:\n")
print(f"Clientes Convertidos: {int(acquired_customer_o_5)}")
print(f"Valor generado: {net_margin_o_5}")
print(f"Costos Incurridos: {campaign_cost_o_5}")
print(f"Beneficios netos generados: {campaign_benefit_o_5}")
print(f"ROI: {campaign_roi_o_5 * 100:.2f}%")

Resumen Optimista de la Campaña 5:

Clientes Convertidos: 1035
Valor generado: 1316413.15
Costos Incurridos: 240805.08000000007
Beneficios netos generados: 1075608.0699999998
ROI: 446.67%


## Campañas Propuestas

A continuación se resumen los resultados potenciales según el escenario, considerando que para la propuesta comercial se considerarán únicamente las campañas 1 y 4 por un tema de simplificación en la ejecución, considerando las campañas 2, 3 y 5 como una segunda etapa de ejecución para concentrar el foco en las primeras propuestas.

Esto consideraría:

-  Una campaña enfocada en la reactivación y volúmen de clientes siendo esta la Capmaña 1 "Reactiva tu Cuenta".
- Una campaña de rentabilización de un segmento específico de clientes siendo la Campaña 4 "Tranquilidad Garantizada".

In [ ]:
campaign_base_total = campaign_base_1 + campaign_base_4
campaign_base_total

217103

In [ ]:
campaign_cost_total = campaign_cost_1 + campaign_cost_4
campaign_cost_total

8326.31

### Escenario Pesimista

In [ ]:
acquired_customer_p_total = acquired_customer_p_1 + acquired_customer_p_4

net_margin_p_total = net_margin_p_1 + net_margin_p_4

campaign_benefit_p_total = campaign_benefit_p_1 + campaign_benefit_p_4

campaign_roi_p_total = campaign_benefit_p_total / (campaign_cost_total + (net_margin_p_total - campaign_benefit_p_total))

campaign_cost_p_total = net_margin_p_total - campaign_benefit_p_total

print(f"Resumen Pesimista de la Campaña General:\n")
print(f"Clientes Contactados: {int(campaign_base_total)}")
print(f"Conversion Rate promedio: {(int(acquired_customer_p_total)/int(campaign_base_total)) * 100:.2f}%")
print(f"Clientes Convertidos: {int(acquired_customer_p_total)}")
print(f"Valor Total generado: {net_margin_p_total}")
print(f"Costos Incurridos: {campaign_cost_p_total}")
print(f"Beneficios Totales netos generados: {campaign_benefit_p_total}")
print(f"ROI: {campaign_roi_p_total * 100:.2f}%")

Resumen Pesimista de la Campaña General:

Clientes Contactados: 217103
Conversion Rate promedio: 0.35%
Clientes Convertidos: 757
Valor Total generado: 632023.2
Costos Incurridos: 119605.10999999999
Beneficios Totales netos generados: 512418.08999999997
ROI: 400.54%


### Escenario Real

Resumen General

In [ ]:
acquired_customer_r_total = acquired_customer_r_1 + acquired_customer_r_4

net_margin_r_total = net_margin_r_1 + net_margin_r_4

campaign_benefit_r_total = campaign_benefit_r_1 + campaign_benefit_r_4

campaign_roi_r_total = campaign_benefit_r_total / (campaign_cost_total + (net_margin_r_total - campaign_benefit_r_total))

campaign_cost_r_total = net_margin_r_total - campaign_benefit_r_total

print(f"Resumen Realista de la Campaña General:\n")
print(f"Clientes Contactados: {int(campaign_base_total)}")
print(f"Conversion Rate promedio: {(int(acquired_customer_r_total)/int(campaign_base_total)) * 100:.2f}%")
print(f"Clientes Convertidos: {int(acquired_customer_r_total)}")
print(f"Costos Incurridos: {campaign_cost_r_total}")
print(f"Valor Total generado: {net_margin_r_total}")
print(f"Beneficios Totales netos generados: {campaign_benefit_r_total}")
print(f"ROI: {campaign_roi_r_total * 100:.2f}%")

Resumen Realista de la Campaña General:

Clientes Contactados: 217103
Conversion Rate promedio: 0.93%
Clientes Convertidos: 2018
Costos Incurridos: 305063.1100000001
Valor Total generado: 1685455.2
Beneficios Totales netos generados: 1380392.0899999999
ROI: 440.47%


Proporción de Beneficios por Campaña

In [ ]:
print(f"Resumen de proporción de Beneficios por Campaña:\n")

print(f"1. Reactiva tu Cuenta: {(campaign_benefit_r_1 / campaign_benefit_r_total) * 100:.2f}%")
print(f"2. Planifica tu Futuro Hoy: {(campaign_benefit_r_2 / campaign_benefit_r_total) * 100:.2f}%")
print(f"3. Rentabilidad con Equilibrio: {(campaign_benefit_r_3 / campaign_benefit_r_total) * 100:.2f}%")
print(f"4. Tranquilidad Garantizada: {(campaign_benefit_r_4 / campaign_benefit_r_total) * 100:.2f}%")
print(f"5. Tu Ahorro Inteligente: {(campaign_benefit_r_5 / campaign_benefit_r_total) * 100:.2f}%")

Resumen de proporción de Beneficios por Campaña:

1. Reactiva tu Cuenta: 5.88%
2. Planifica tu Futuro Hoy: 93.72%
3. Rentabilidad con Equilibrio: 13.60%
4. Tranquilidad Garantizada: 94.12%
5. Tu Ahorro Inteligente: 41.27%


Proporción de Adquisiciones por Campaña

In [ ]:
print(f"Resumen de proporción de Conversiones por Campaña:\n")

print(f"1. Reactiva tu Cuenta: {(acquired_customer_r_1 / acquired_customer_r_total) * 100:.2f}%")
print(f"4. Tranquilidad Garantizada: {(acquired_customer_r_4 / acquired_customer_r_total) * 100:.2f}%")

Resumen de proporción de Conversiones por Campaña:

1. Reactiva tu Cuenta: 84.54%
4. Tranquilidad Garantizada: 15.46%


Costos y ROI  por Campaña

In [ ]:
print(f"Costos y ROI esperados por Campaña:\n")

print(f"1. Reactiva tu Cuenta:          ---> Gastos: {int(campaign_cost_r_1)}€ ---> ROI: {campaign_roi_r_1 * 100:.2f}%")
print(f"2. Planifica tu Futuro Hoy:     ---> Gastos: {int(campaign_cost_r_2)}€ ---> ROI: {campaign_roi_r_2 * 100:.2f}%")
print(f"3. Rentabilidad con Equilibrio: ---> Gastos: {int(campaign_cost_r_3)}€ ---> ROI: {campaign_roi_r_3 * 100:.2f}%")
print(f"4. Tranquilidad Garantizada:    ---> Gastos: {int(campaign_cost_r_4)}€ ---> ROI: {campaign_roi_r_4 * 100:.2f}%")
print(f"5. Tu Ahorro Inteligente:       ---> Gastos: {int(campaign_cost_r_5)}€ ---> ROI: {campaign_roi_r_5 * 100:.2f}%")

Costos y ROI esperados por Campaña:

1. Reactiva tu Cuenta:          ---> Gastos: 19512€ ---> ROI: 415.66%
2. Planifica tu Futuro Hoy:     ---> Gastos: 284650€ ---> ROI: 454.48%
3. Rentabilidad con Equilibrio: ---> Gastos: 44754€ ---> ROI: 419.50%
4. Tranquilidad Garantizada:    ---> Gastos: 285550€ ---> ROI: 455.01%
5. Tu Ahorro Inteligente:       ---> Gastos: 132347€ ---> ROI: 726.20%


### Escenario Optimista

In [ ]:
acquired_customer_o_total = acquired_customer_o_1 + acquired_customer_o_4

net_margin_o_total = net_margin_o_1 + net_margin_o_4

campaign_benefit_o_total = campaign_benefit_o_1 + campaign_benefit_o_4

campaign_roi_o_total = campaign_benefit_o_total / (campaign_cost_total + (net_margin_o_total - campaign_benefit_o_total))

campaign_cost_o_total = net_margin_o_total - campaign_benefit_o_total

print(f"Resumen Optimista de la Campaña General:\n")
print(f"Clientes Contactados: {int(campaign_base_total)}")
print(f"Conversion Rate promedio: {(int(acquired_customer_o_total)/int(campaign_base_total)) * 100:.2f}%")
print(f"Clientes Convertidos: {int(acquired_customer_o_total)}")
print(f"Valor Total generado: {net_margin_o_total}")
print(f"Costos Incurridos: {campaign_cost_o_total}")
print(f"Beneficios Totales netos generados: {campaign_benefit_o_total}")
print(f"ROI: {campaign_roi_o_total * 100:.2f}%")

Resumen Optimista de la Campaña General:

Clientes Contactados: 217103
Conversion Rate promedio: 1.74%
Clientes Convertidos: 3783
Valor Total generado: 3160286.0
Costos Incurridos: 564700.31
Beneficios Totales netos generados: 2595585.69
ROI: 452.96%
